In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import igraph as ig
import numpy as np
import pandas as pd
import src.analysis.robustness_testing as rt
import src.analysis.community_aware_centrality as cac
from src.visualization import visualize

# import matplotlib.pyplot as plt
# %matplotlib inline

In [14]:
g = ig.Graph.Read_Picklez('../data/processed/roadnet-PA.pkl.gz')
part = g.community_leiden(objective_function='modularity')

In [15]:
centralities = [cac.modularity_vitality,
                cac.absolute_modularity_vitality,
                cac.adjusted_modular_centrality_degree,
                cac.masuda,
                cac.community_hub_bridge,
                cac.weighted_modular_centrality_degree,
                cac.degree]

In [16]:
abbreviations = {'modularity_vitality': 'MV',
                'absolute_modularity_vitality': 'AMV',
                'adjusted_modular_centrality_degree': 'AMC-D',
                'masuda': 'Mas',
                'community_hub_bridge': 'CHB',
                'weighted_modular_centrality_degree': 'WMC-D',
                'degree': 'Deg'}

In [ ]:
sigma_results = {}
value_results = {}
for centrality_func in centralities:
    values, rho, sigma, rho_e = rt.initial_attack(g, part, centrality_func,
                                                 calculations=100)
    sigma_results[centrality_func.__name__] = [rho, sigma, rho_e]
    value_results[centrality_func.__name__] = values
    print(f'finished {centrality_func.__name__}')

finished modularity_vitality
finished absolute_modularity_vitality
finished adjusted_modular_centrality_degree
finished masuda


In [ ]:
for function, (rho, sigma, rho_e) in sigma_results.items():
    node_cost = np.trapz(y=sigma, x=rho)
    edge_cost = np.trapz(y=sigma, x=rho_e)
    print(f'{abbreviations[function]}:')
    print(f'node cost: {node_cost}, edge cost: {edge_cost}')

In [ ]:
visualize.fragFigs('PA Roads', sigma_results, abbrv=abbreviations, legend=True)

In [ ]:
visualize.heatmap('PA Roads', value_results, abbrv=abbreviations)